In [2]:
import osmnx as ox

In [3]:
#loading buildings from Södermalm in Stockholm
tags = {"building": True}
gdf = ox.geometries_from_place("Södermalm, Stockholm", tags)

In [4]:
#choosing only the element_type "way", which perhaps(!) is the correct element_type
gdf = gdf.loc[(gdf.index.get_level_values('element_type') == 'way')]

In [5]:
#according to instructions from osm nx-docs, making correct projection
gdf_proj = ox.project_gdf(gdf)

In [6]:
#using geopandas to calculate areas from the geometric polygons
gdf_proj['area'] = gdf_proj.area

In [7]:
#keeping only buildings with level information
gdf_l = gdf_proj.dropna(subset=['building:levels'])

In [8]:
gdf_l['height'] = gdf_l['height'].astype('float')

/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [9]:
gdf_l['building:levels'] = gdf_l['building:levels'].astype('float')

In [10]:
#calculating floor area
gdf_l['floor'] = gdf_l['building:levels'] * gdf_l['area']

/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [11]:
gdf_l['floor']

element_type  osmid     
way           7266905        4718.943831
              17024283      76126.727387
              17024436       5316.839721
              17024460        989.224281
              17392811      55486.788250
                                ...     
              1030387591     5820.197621
              1065362442       71.449469
              1065843681       43.531342
              1065843683      251.360474
              1121465295       30.872853
Name: floor, Length: 516, dtype: float64

In [12]:
#using the buildings with both height and level info to get an estimate of median level height
heights = gdf_l.loc[gdf_l['height'] > 0]['height'] / gdf_l.loc[gdf_l['height'] > 0]['building:levels'] 
heights

element_type  osmid    
way           7266905      6.250000
              37046494     3.866667
              37299689     3.583333
              42672700     3.291667
              149948736    3.571429
              846741207    4.131579
              846742152    8.100000
dtype: float64

In [13]:
#using the median of heights lower than the 90% percentile (clearing out messy values)
levelheight = heights.loc[heights < heights.quantile(0.9)].median()

In [14]:
levelheight

3.725

In [15]:
#the levelheight could be use to estimate level-amounts on buildings with only height information, 
#but in this case all buildings with height information had level information also.

In [19]:
#multiplying floor area with figure from Antti's model, using climate zone 5 for Stockholm
gdf_l['kWh/a'] = gdf_l['floor'] * 159.993

/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [21]:
#using this article https://www.sei.org/publications/swedish-heat-energy-system-new-tensions-and-lock-ins-after-a-successful-transition/#:~:text=Fossil%20fuel%20energy%20has%20been,demand%20for%20heating%20in%20buildings.
#to calculate an estimate of the energy mix and it's co2-emissions using Antti's model
co2 = 0.5*0.195 + 0.25*0.104 + 0.08*0.255 + 0.08*0.354 + 0.09*0.403
co2

0.20848999999999998

In [22]:
#multiplying kWh with co2/kWh for estimate of co2-emissions
gdf_l['co2/a'] = gdf_l['kWh/a'] * co2

/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
